# Feature Database

In [123]:
import numpy as np
import pandas as pd
from datetime import datetime
import yfinance as yf
from fredapi import Fred
import ta

## Common Features

In [124]:
start_date = datetime(2020, 1, 1)
end_date = datetime(2024, 8, 31)
NFLX_STOCK = yf.download("NFLX", start_date, end_date)
NFLX_STOCK.columns = NFLX_STOCK.columns.droplevel(1)
NFLX_STOCK.head()

[*********************100%***********************]  1 of 1 completed


Price,Adj Close,Close,High,Low,Open,Volume
Date,,,,,,
2020-01-02,329.809998,329.809998,329.980011,324.779999,326.100006,4485800
2020-01-03,325.899994,325.899994,329.859985,325.529999,326.779999,3806900
2020-01-06,335.829987,335.829987,336.359985,321.200012,323.119995,5663100
2020-01-07,330.750000,330.750000,336.700012,330.299988,336.470001,4703200
2020-01-08,339.260010,339.260010,342.700012,331.049988,331.489990,7104500


## FRED

In [125]:
# Japanese Yen to U.S. Dollar Spot Exchange Rate
# U.S. Dollars to Euro Spot Exchange Rate
# Coinbase Bitcoin
fred = Fred(api_key='7b9f7e9c637e4f731da1f888d14b6037')
feat_list = ["SP500", "DEXJPUS", "DEXUSEU", "CBBTCUSD"]
feat_df = pd.DataFrame()
for feat in feat_list:
    feature = fred.get_series(feat, start_date, end_date)
    feature = feature.to_frame(feat)
    feature.dropna(inplace=True)
    feat_df = pd.concat([feat_df, feature], axis=1)
feat_df.dropna(inplace=True)
feat_df.index.name = "Date"
feat_df

,SP500,DEXJPUS,DEXUSEU,CBBTCUSD
Date,,,,
2020-01-02,3257.85,108.43,1.1166,6945.01
2020-01-03,3234.85,107.94,1.1173,7335.54
2020-01-06,3246.28,108.36,1.1187,7773.48
2020-01-07,3237.18,108.53,1.1138,8169.95
2020-01-08,3253.05,109.03,1.1117,8035.08
...,...,...,...,...
2024-08-26,5616.84,144.51,1.1164,62838.55
2024-08-27,5625.80,144.26,1.1160,59398.77
2024-08-28,5592.18,144.47,1.1127,59024.00


In [126]:
NFLX_STOCK = pd.concat([NFLX_STOCK, feat_df], axis=1)
NFLX_STOCK.head()

,Adj Close,Close,High,Low,Open,Volume,SP500,DEXJPUS,DEXUSEU,CBBTCUSD
Date,,,,,,,,,,
2020-01-02,329.809998,329.809998,329.980011,324.779999,326.100006,4485800,3257.85,108.43,1.1166,6945.01
2020-01-03,325.899994,325.899994,329.859985,325.529999,326.779999,3806900,3234.85,107.94,1.1173,7335.54
2020-01-06,335.829987,335.829987,336.359985,321.200012,323.119995,5663100,3246.28,108.36,1.1187,7773.48
2020-01-07,330.750000,330.750000,336.700012,330.299988,336.470001,4703200,3237.18,108.53,1.1138,8169.95
2020-01-08,339.260010,339.260010,342.700012,331.049988,331.489990,7104500,3253.05,109.03,1.1117,8035.08


## Fama-French

In [127]:
data_ff5 = pd.read_csv('resources/F-F_Research_Data_5_Factors_2x3_daily.csv')
data_ff5.rename(columns={"Unnamed: 0": "Date"}, inplace=True)
data_ff5["Date"] = pd.to_datetime(data_ff5["Date"], format="%Y%m%d")
data_ff5 = data_ff5[(data_ff5["Date"] >= start_date) & (data_ff5["Date"] <= end_date)]
data_ff5 = data_ff5.set_index('Date')
data_ff5

,Mkt-RF,SMB,HML,RMW,CMA,RF
Date,,,,,,
2020-01-02,0.86,-0.97,-0.33,0.24,-0.22,0.006
2020-01-03,-0.67,0.30,0.00,-0.13,-0.11,0.006
2020-01-06,0.36,-0.21,-0.55,-0.17,-0.26,0.006
2020-01-07,-0.19,-0.03,-0.25,-0.13,-0.24,0.006
2020-01-08,0.47,-0.16,-0.66,-0.16,-0.18,0.006
...,...,...,...,...,...,...
2024-08-26,-0.34,0.33,0.17,0.13,-0.06,0.022
2024-08-27,0.05,-0.90,0.02,0.27,0.23,0.022
2024-08-28,-0.67,-0.22,1.14,0.55,-0.16,0.022


In [128]:
NFLX_STOCK = pd.concat([NFLX_STOCK, data_ff5], axis=1)
NFLX_STOCK.head()

,Adj Close,Close,High,Low,Open,Volume,SP500,DEXJPUS,DEXUSEU,CBBTCUSD,Mkt-RF,SMB,HML,RMW,CMA,RF
Date,,,,,,,,,,,,,,,,
2020-01-02,329.809998,329.809998,329.980011,324.779999,326.100006,4485800,3257.85,108.43,1.1166,6945.01,0.86,-0.97,-0.33,0.24,-0.22,0.006
2020-01-03,325.899994,325.899994,329.859985,325.529999,326.779999,3806900,3234.85,107.94,1.1173,7335.54,-0.67,0.30,0.00,-0.13,-0.11,0.006
2020-01-06,335.829987,335.829987,336.359985,321.200012,323.119995,5663100,3246.28,108.36,1.1187,7773.48,0.36,-0.21,-0.55,-0.17,-0.26,0.006
2020-01-07,330.750000,330.750000,336.700012,330.299988,336.470001,4703200,3237.18,108.53,1.1138,8169.95,-0.19,-0.03,-0.25,-0.13,-0.24,0.006
2020-01-08,339.260010,339.260010,342.700012,331.049988,331.489990,7104500,3253.05,109.03,1.1117,8035.08,0.47,-0.16,-0.66,-0.16,-0.18,0.006


## ADS

In [129]:
ads = pd.read_excel("resources/ADS_Index_Most_Current_Vintage.xlsx")
ads.rename(columns={"Unnamed: 0": "Date"}, inplace=True)
ads["Date"] = pd.to_datetime(ads["Date"], format="%Y:%m:%d")
ads = ads[(ads["Date"] >= start_date) & (ads["Date"] <= end_date)]
ads = ads.set_index("Date")
ads

,ADS_Index
Date,
2020-01-01,-0.301644
2020-01-02,-0.289801
2020-01-03,-0.278009
2020-01-04,-0.266263
2020-01-05,-0.254558
...,...
2024-08-27,-0.114812
2024-08-28,-0.119569
2024-08-29,-0.125109


In [130]:
NFLX_STOCK = pd.concat([NFLX_STOCK, ads], axis=1)
NFLX_STOCK.dropna()

,Adj Close,Close,High,Low,Open,Volume,SP500,DEXJPUS,DEXUSEU,CBBTCUSD,Mkt-RF,SMB,HML,RMW,CMA,RF,ADS_Index
Date,,,,,,,,,,,,,,,,,
2020-01-02,329.809998,329.809998,329.980011,324.779999,326.100006,4485800.0,3257.85,108.43,1.1166,6945.01,0.86,-0.97,-0.33,0.24,-0.22,0.006,-0.289801
2020-01-03,325.899994,325.899994,329.859985,325.529999,326.779999,3806900.0,3234.85,107.94,1.1173,7335.54,-0.67,0.30,0.00,-0.13,-0.11,0.006,-0.278009
2020-01-06,335.829987,335.829987,336.359985,321.200012,323.119995,5663100.0,3246.28,108.36,1.1187,7773.48,0.36,-0.21,-0.55,-0.17,-0.26,0.006,-0.243161
2020-01-07,330.750000,330.750000,336.700012,330.299988,336.470001,4703200.0,3237.18,108.53,1.1138,8169.95,-0.19,-0.03,-0.25,-0.13,-0.24,0.006,-0.232068
2020-01-08,339.260010,339.260010,342.700012,331.049988,331.489990,7104500.0,3253.05,109.03,1.1117,8035.08,0.47,-0.16,-0.66,-0.16,-0.18,0.006,-0.221273
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-26,688.440002,688.440002,690.590027,681.640015,687.260010,1354200.0,5616.84,144.51,1.1164,62838.55,-0.34,0.33,0.17,0.13,-0.06,0.022,-0.110836
2024-08-27,695.719971,695.719971,707.890015,686.919983,688.530029,3164900.0,5625.80,144.26,1.1160,59398.77,0.05,-0.90,0.02,0.27,0.23,0.022,-0.114812
2024-08-28,683.840027,683.840027,696.669983,677.099976,695.830017,2430600.0,5592.18,144.47,1.1127,59024.00,-0.67,-0.22,1.14,0.55,-0.16,0.022,-0.119569


## Self-Engineered Feature

In [131]:
NFLX_STOCK["Typical_Price"] = NFLX_STOCK[["High", "Low", "Close"]].mean(axis=1)
NFLX_STOCK["Typical_Price_Return"] = (
    np.log(NFLX_STOCK.Typical_Price) - np.log(NFLX_STOCK.Typical_Price.shift(+1))
) * 100.0

In [132]:
NFLX_STOCK.dropna()

,Adj Close,Close,High,Low,Open,Volume,SP500,DEXJPUS,DEXUSEU,CBBTCUSD,Mkt-RF,SMB,HML,RMW,CMA,RF,ADS_Index,Typical_Price,Typical_Price_Return
Date,,,,,,,,,,,,,,,,,,,
2020-01-03,325.899994,325.899994,329.859985,325.529999,326.779999,3806900.0,3234.85,107.94,1.1173,7335.54,-0.67,0.30,0.00,-0.13,-0.11,0.006,-0.278009,327.096659,-0.333699
2020-01-07,330.750000,330.750000,336.700012,330.299988,336.470001,4703200.0,3237.18,108.53,1.1138,8169.95,-0.19,-0.03,-0.25,-0.13,-0.24,0.006,-0.232068,332.583333,0.437942
2020-01-08,339.260010,339.260010,342.700012,331.049988,331.489990,7104500.0,3253.05,109.03,1.1117,8035.08,0.47,-0.16,-0.66,-0.16,-0.18,0.006,-0.221273,337.670003,1.517864
2020-01-09,335.660004,335.660004,343.420013,334.609985,342.000000,4709300.0,3274.70,109.47,1.1106,7813.78,0.65,-0.71,-0.48,-0.14,0.04,0.006,-0.210773,337.896667,0.067103
2020-01-10,329.049988,329.049988,338.500000,327.269989,337.130005,4718300.0,3265.35,109.50,1.1119,8183.89,-0.34,-0.27,-0.33,0.04,-0.08,0.006,-0.200564,331.606659,-1.879063
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-23,686.729980,686.729980,696.070007,678.630005,694.130005,2354500.0,5634.61,144.86,1.1176,64130.87,1.29,1.90,0.85,-0.48,0.68,0.022,-0.103443,687.143331,-0.738019
2024-08-27,695.719971,695.719971,707.890015,686.919983,688.530029,3164900.0,5625.80,144.26,1.1160,59398.77,0.05,-0.90,0.02,0.27,0.23,0.022,-0.114812,696.843323,1.438641
2024-08-28,683.840027,683.840027,696.669983,677.099976,695.830017,2430600.0,5592.18,144.47,1.1127,59024.00,-0.67,-0.22,1.14,0.55,-0.16,0.022,-0.119569,685.869995,-1.587250


## Returns (Y target)

In [133]:
NFLX_STOCK["Returns"] = NFLX_STOCK["Adj Close"] - NFLX_STOCK["Adj Close"].shift(1) # difference
NFLX_STOCK["Daily_Return"] = np.log(NFLX_STOCK["Adj Close"]) - np.log(NFLX_STOCK["Adj Close"].shift(1)) # percentage
NFLX_STOCK.head()

,Adj Close,Close,High,Low,Open,Volume,SP500,DEXJPUS,DEXUSEU,CBBTCUSD,...,SMB,HML,RMW,CMA,RF,ADS_Index,Typical_Price,Typical_Price_Return,Returns,Daily_Return
Date,,,,,,,,,,,,,,,,,,,,,
2020-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-0.301644,NaN,NaN,NaN,NaN
2020-01-02,329.809998,329.809998,329.980011,324.779999,326.100006,4485800.0,3257.85,108.43,1.1166,6945.01,...,-0.97,-0.33,0.24,-0.22,0.006,-0.289801,328.190002,NaN,NaN,NaN
2020-01-03,325.899994,325.899994,329.859985,325.529999,326.779999,3806900.0,3234.85,107.94,1.1173,7335.54,...,0.30,0.00,-0.13,-0.11,0.006,-0.278009,327.096659,-0.333699,-3.910004,-0.011926
2020-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-0.266263,NaN,NaN,NaN,NaN
2020-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-0.254558,NaN,NaN,NaN,NaN


## Generate Dataset

In [134]:
filename = f"NFLX_feature_mart.csv"
NFLX_STOCK.dropna(inplace=True)
NFLX_STOCK.head()

,Adj Close,Close,High,Low,Open,Volume,SP500,DEXJPUS,DEXUSEU,CBBTCUSD,...,SMB,HML,RMW,CMA,RF,ADS_Index,Typical_Price,Typical_Price_Return,Returns,Daily_Return
Date,,,,,,,,,,,,,,,,,,,,,
2020-01-03,325.899994,325.899994,329.859985,325.529999,326.779999,3806900.0,3234.85,107.94,1.1173,7335.54,...,0.30,0.00,-0.13,-0.11,0.006,-0.278009,327.096659,-0.333699,-3.910004,-0.011926
2020-01-07,330.750000,330.750000,336.700012,330.299988,336.470001,4703200.0,3237.18,108.53,1.1138,8169.95,...,-0.03,-0.25,-0.13,-0.24,0.006,-0.232068,332.583333,0.437942,-5.079987,-0.015242
2020-01-08,339.260010,339.260010,342.700012,331.049988,331.489990,7104500.0,3253.05,109.03,1.1117,8035.08,...,-0.16,-0.66,-0.16,-0.18,0.006,-0.221273,337.670003,1.517864,8.510010,0.025404
2020-01-09,335.660004,335.660004,343.420013,334.609985,342.000000,4709300.0,3274.70,109.47,1.1106,7813.78,...,-0.71,-0.48,-0.14,0.04,0.006,-0.210773,337.896667,0.067103,-3.600006,-0.010668
2020-01-10,329.049988,329.049988,338.500000,327.269989,337.130005,4718300.0,3265.35,109.50,1.1119,8183.89,...,-0.27,-0.33,0.04,-0.08,0.006,-0.200564,331.606659,-1.879063,-6.610016,-0.019889


In [135]:
NFLX_STOCK.to_csv(filename, index=True)